In [1]:
# Import necessary packages
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

import os
#For K-nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import argparse

class DatasetLoader:
    def load(self, image_paths, verbose=-1):

        # Initialize the list of images and labels
        data = []
        labels = []

        # Loop over input paths to read the data
        for (i, path) in enumerate(image_paths):
            # Load images
            # Assuming path in following format            # /path/to/dataset/{class}/{image-name}.jpg
            image = cv.imread(path)
            label = path.split(os.path.sep)[-2]

            # Resize image
            image = cv.resize(image, (128, 128))
            
            orb = cv.ORB_create()
            # find the keypoints with ORB
            kp = orb.detect(image,None)
            # compute the descriptors with ORB
            kp, des = orb.compute(image, kp)
            # draw only keypoints location,not size and orientation
            image = cv.drawKeypoints(image, kp, None, color=(0,255,0), flags=0)

            # Push into data list
            data.append(image)
            labels.append(label)

            # Show update
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1, len(image_paths)))

        # Return a tuple of data and labels
        return (np.array(data), np.array(labels))

#Get link dataset    
train_path="animals_dataset/animals"
class_names=os.listdir(train_path)

def img_list(path):
    return (os.path.join(path,f) for f in os.listdir(path))
image_paths=[]
for training_name in class_names:
    dir_=os.path.join(train_path,training_name)
    class_path=img_list(dir_)
    image_paths+=class_path
    
#Load dataset
print("[INFO] Loading images")
sdl = DatasetLoader()
(data, labels) = sdl.load(image_paths, verbose=500)


#Convert to vector
data = data.reshape((data.shape[0], 128*128*3))

#size of data
print("[INFO] features matrix: {:.1f}MB".format(
    data.nbytes / (1024 * 1024.0)
))

#Convert label to number

le = LabelEncoder()
labels = le.fit_transform(labels)

#train and test
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

#evaluate K-nn
print("[INFO] evaluating k-NN classifier")

model = KNeighborsClassifier(n_neighbors=35)
model.fit(trainX, trainY)
print(classification_report(testY, model.predict(testX), target_names=le.classes_))



[INFO] Loading images
[INFO] processed 500/3000
[INFO] processed 1000/3000
[INFO] processed 1500/3000
[INFO] processed 2000/3000
[INFO] processed 2500/3000
[INFO] processed 3000/3000
[INFO] features matrix: 140.6MB
[INFO] evaluating k-NN classifier
              precision    recall  f1-score   support

        cats       0.51      0.64      0.56       249
        dogs       0.44      0.61      0.51       262
       panda       0.97      0.27      0.42       239

    accuracy                           0.51       750
   macro avg       0.64      0.51      0.50       750
weighted avg       0.63      0.51      0.50       750



In [2]:
import random
import time


import tkinter
import tkinter as tk
from tkinter import *

from PIL import ImageTk, Image, ImageFilter


#Set environment 
os.environ.__setitem__('DISPLAY', ':0.0')


root = Tk()
root.title("Game")
root.minsize(600, 400)






# result = Tk()
# result.minsize(500, 300)



human_point = 0
bot_point = 0

human_answer = -1
bot_answer = -1

#Flag
flag = 0
boom = 0

#image index
i = 0
i = random.randint(1, 3000)

#Clock in UI
def boom_out():
    global boom
    if boom == 1:
        root.destroy()
        result = Tk()
        result.title('Result')
        result_text = "Time out"

        result_up = tk.Label(text= result_text)
        result_up.place(x = 20, y = 0)

        result_text2 = "YOU LOSE"
        result_down = tk.Label(text= result_text2)
        result_down.place(x = 20, y = 30)

        result.mainloop()

        print("out")
        os._exit(1)  
        
def clock2(string, second = 1):
    global lotifi_wait
    global lotifi_second
    notifi_wait['text'] = string
    
    notifi_second['text'] = str(second)
    next_time = second - 1
    if next_time > -1 and boom == 1:
        root.after(1000, lambda: clock2(string, next_time))
    else:
        notifi_wait['text'] = ""
        notifi_second['text'] = ""
        
def clock(string, second = 1):
    global lotifi_wait
    global lotifi_second
    global lotifi_answer
    global bot_lotifi_answer
    global flag
    global boom
    
    notifi_wait['text'] = string
    
    notifi_second['text'] = str(second)
    next_time = second - 1
    if next_time > -1:
        root.after(1000, lambda: clock(string, next_time))
    else:
        notifi_wait['text'] = "The next question, please choose!!!"
        notifi_second['text'] = ""
        notifi_answer['text']=""
        bot_notifi_answer['text']=""
        show_image_blur()
        flag = 0
        boom = 1
        root.after(7000, lambda: boom_out())
        
        
        
        clock2("time remain", 7)
        
        
        
        
        

        
        

def choose_answer(animal_name):
    global human_point
    global bot_point
    global notifi_answer
    global bot_lotifi_answer
    global flag
    global i
    global boom
    
    
    #Exit boom out
    boom = 0
    
    
    if(flag != 0):
        return
        
    flag = 1

    if animal_name == "cat":
        human_answer = 0
    if animal_name == "dog":
        human_answer = 1
    if animal_name == "panda":
        human_answer = 2
    show_image()
    if human_answer == labels[i]:
        #Check end the game
        
            
        notifi_answer['text'] = "Correct!!!"
        human_point += 1
        if human_point == 5 or bot_point > 4:
            root.destroy()
            result = Tk()
            result.title('Result')
            result_text = "Result: You " + str(human_point) + " Bot: " + str(bot_point)
            
            result_up = tk.Label(text= result_text)
            result_up.place(x = 20, y = 0)
            
            if human_point > bot_point:
                result_text2 = "YOU WIN"
            elif human_point < bot_point:
                result_text2 = "YOU LOSE"
            else: 
                result_text2 = "()-()"
            result_down = tk.Label(text= result_text2)
            result_down.place(x = 20, y = 30)
            
            result.mainloop()
            
            print("out")
            os._exit(1)        
#             result.mainloop()

        
        human_point_bt = tk.Label(text= str(human_point))
        human_point_bt.place(x = 20, y = 100)

        
    else:
        notifi_answer['text'] = "Wrong!!!"
        
    if model.predict([data[i]])[0] == labels[i]:
        bot_notifi_answer['text'] = "Correct!!!"
        bot_point += 1
        

        bot_point_bt = tk.Label(text= str(bot_point))
        bot_point_bt.place(x =500, y = 100)
    else:
        bot_notifi_answer['text'] = "Wrong!!!"
    
    
        
    clock("Please wait...", 3)
        
    
        
    
#Notifi answer and wait notifi
notifi_answer=tkinter.Label(text="")
notifi_answer.place(x=20, y=300)

bot_notifi_answer=tkinter.Label(text="Ready")
bot_notifi_answer.place(x=450, y=350)

notifi_wait=tkinter.Label(text="")
notifi_wait.place(x=295, y=300)

notifi_second=tkinter.Label(text="")
notifi_second.place(x=295,y=320)
        
#Human_point
human_point_bt = tk.Label(text= str(human_point))
human_point_bt.place(x = 20, y = 100)
human_point_text_bt = tk.Label(text= "Human point")
human_point_text_bt.place(x = 20, y = 50)

bot_point_bt = tk.Label(text= str(bot_point))
bot_point_bt.place(x =500, y = 100)
bot_point_text_bt = tk.Label(text= "Bot point")
bot_point_text_bt.place(x = 500, y = 50)
#Bot image
bot_image = Image.open("bot.jpg")
    
newsize = (70, 60)
bot_image = bot_image.resize(newsize)
    
bot_image = ImageTk.PhotoImage(bot_image)
label2 = tkinter.Label(image=bot_image)
label2.image = bot_image
label2.place(x=500, y=350)


#Image
def show_image():
    global i
    
    image = Image.open(image_paths[i])
    
    newsize = (350, 300)
    image = image.resize(newsize)
    
    image = ImageTk.PhotoImage(image)
    label1 = tkinter.Label(image=image)
    label1.image = image
    label1.place(x=120, y=0)

def show_image_blur():
    global i
    i = random.randint(1, 3000)

    image = Image.open(image_paths[i])
    newsize = (350, 300)
    image = image.resize(newsize)
    image_blur = image.filter(ImageFilter.GaussianBlur(radius=10))
    image_blur = ImageTk.PhotoImage(image_blur)
    
    label1 = tkinter.Label(image=image_blur)
    label1.image = image_blur
    label1.place(x=120, y=0)

#Start button
start = tk.Button(text="START", command = show_image_blur)
start.place(x = 295, y = 100)



#Choose Button
dog = tk.Button(text="DOG", command = lambda: choose_answer("dog"))
dog.place(x = 0, y = 330)

cat = tk.Button(text="CAT", command = lambda: choose_answer("cat"))
cat.place(x = 70, y = 330)

panda = tk.Button(text="PANDA", command = lambda: choose_answer("panda"))
panda.place(x = 140, y = 330)

# End

    
    


root.mainloop()
